## Дерево решений

Задание
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn (это возможно, так как мы реализуем только малую часть функциональности)
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала так же или качественнее, чем у дерева из sklearn
5. Применить реализованное дерево решений для задачи Titanic на kaggle. Применить для той же задачи дерево решений из sklearn. Применить кросс-валидацию для подбора параметров. Сравнить с результатами предыдущих моделей. Если результат улучшился - сделать сабмит. Написать отчет о результатах.

In [274]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

реализация для понимания неопределенности gini

In [275]:
def impurity_gini(node_classes):
        cls_cnt = np.bincount(node_classes)
        print(cls_cnt)
        gini = 1 - ((cls_cnt/node_classes.shape[0])**2).sum()
        return gini

def entropy(node_classes):
    cls_cnt = np.bincount(node_classes)
    all_cnt = node_classes.shape[0]
    p = cls_cnt/all_cnt
    entropy = - np.sum(p*np.log2(p))
    return entropy

def misclass(node_classes):
    cls_cnt = np.bincount(node_classes)
    all_cnt = node_classes.shape[0]
    p = cls_cnt/all_cnt
    misclass = 1 - p.max()
    return misclass

classes = np.array([0,0,0,0,0,0,0,0,0,0])
print(impurity_gini(classes))
print(entropy(classes))
print(misclass(classes))


[10]
0.0
-0.0
0.0


In [276]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger('decision tree')
logger.setLevel(logging.INFO)

In [277]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=10, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        # на сколько частей делим
        self.min_samples_split = min_samples_split
        # максимальная глубина
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        
        # функция Information Gain
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            logger.debug('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            logger.debug('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        p_l = l_s/(l_s + r_s)
        p_r = r_s/(l_s + r_s)
        gini_parent = 1- (((l_c + r_c)/(l_s + r_s))**2).sum(axis=1)
        logger.debug(['p_l, p_r', p_l, p_r])
        gini_left = 1-((l_c/l_s)**2).sum(axis=1)
        gini_right = 1-((r_c/r_s)**2).sum(axis=1)
        logger.debug(['gini left',  gini_left])
        logger.debug(['gini right',  gini_right])
        logger.debug(['gini parent',  gini_parent])
        logger.debug(['p_l, p_r',  p_l, p_r])
        IG = gini_parent - p_l.T*gini_left - p_r.T*gini_right
        logger.debug(IG)
        return IG.T
       
    def __entropy(self, l_c, l_s, r_c, r_s):
        return # Ваш код 

    def __misclass(self, l_c, l_s, r_c, r_s):
        return # Ваш код

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(sqrt(n_feature))]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        if len(x) != len(y):
            print([x.shape, y.shape])
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        try:
            left_mask = x[:, feature_id] > threshold
            right_mask = ~left_mask
        except:
            print([x[:, feature_id], threshold])
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        #получаем значения x и y  отсортированных по x - под x  подразумевается одна фича
        # сортировка нужна чтобы одинаковые слуовия х оказались рядом
        sorted_x, sorted_y = self.__sort_samples(x, y)
        # считаем число уникальных y - наших классов
        class_number = np.unique(y).shape[0]
        
        logger.debug(['sorted_x = ', sorted_x])
        logger.debug(['sorted_y = ', sorted_y])        
        logger.debug(['class number ', class_number])
        
        """
        
        # выделяем кусок  меж границами с минимальным кол-вом разбиений
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        # нахожим границы переходов между классами в массиве y
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        """
        """
        разбиения не происходит при массиве из 4х элементов. например [1,1,1,0]
        т.к. я не понял почему автор не хочет разбивать такие куски, задам свой кусок кода
        """
        splitted_sorted_y = sorted_y
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + 1
        
        logger.debug(['splitted_sorted_y = ', splitted_sorted_y])
        logger.debug(['r_border_ids = ', r_border_ids])
        
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # считаем кол-во одинаковых элементов  и получаем массив с указанием кол-ва эл]ементов превалирующего класса
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)

        logger.debug(['eq_el_count=', eq_el_count])
        logger.debug(['one_hot_code', one_hot_code])
        logger.debug(['class_increments', class_increments])
        
        
        # рассчитываем и получаем массивы с указанием класса и кол-ва элементов с каждой стороны
        # и кол-во элементов с каждой стороны
        
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes
        
        logger.debug('l_class_count, r_class_count, l_sizes, r_sizes')
        logger.debug([l_class_count, r_class_count, l_sizes, r_sizes])

        # находим наилучший IG для разделения
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
        logger.debug(['gs', gs])
        logger.debug(['idx, gsp[idx]:', idx, gs[idx]])
    
        # возвращаем найденный GS и условия для разделение (среднее значение между граничными элементами)
        left_el_id = l_sizes[idx][0]
        logger.debug(['left_el_id', left_el_id])
        logger.debug(['returned', (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0])
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    
    def debug_threshold(self, x, y):
        self.__find_threshold(x, y)
        
        
    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE
        

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split
        
        #if len(y) < self.min_samples_split:
            
            
        
        
        # если все у одного класса, считаем ноду листом
        if np.unique(y).shape[0] == 1:
            self.tree[node_id] = (self.LEAF_TYPE, y[0], 0)
        elif self.max_depth == depth:
            #print(node_id)
            #print(y)
            y_class = np.unique(y)[np.bincount(y).argmax()]
            self.tree[node_id] = (self.LEAF_TYPE, y_class, 0)
        else: # находим точку ветвления
            
            feature_threshold = []
            
            # для каждого признака находим варианты разбиения и сохраняем в единый массив
            for idx in self.get_feature_ids(x.shape[1]):
                ig, cond = self.__find_threshold(x[:,idx], y)
                if cond is not None:
                    feature_threshold.append([ig, cond, idx])
            # находим строку с наибольшим IG
            if feature_threshold == []:
                print([x, y])
            feature_threshold = np.array(feature_threshold)
            ft_row = feature_threshold[:,0].argmax()
            ig, threshold, feature_id = feature_threshold[ft_row]
            
            # сохраняем ноду
            self.tree[node_id] = (self.NON_LEAF_TYPE, feature_id, threshold)
            # разбиваем
            if threshold is None:
                print(feature_threshold)
                print(self.tree[node_id])
            x_left, x_right, y_left, y_right = self.__div_samples(x, y, feature_id, threshold) 
            if len(x_left)!=len(y_left) or len(x_right)!=len(y_right):
                print([x_left.shape, x_right.shape, y_left.shape, y_right.shape])
            
            # повторяем для каждой ветви
            if len(y_left) > 0:
                self.__fit_node(x_left, y_left, 2 * node_id + 1, depth+1)
            if len(y_right) > 0:
                self.__fit_node(x_right, y_right, 2 * node_id + 2, depth+1)
            
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [278]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


Подготовим датасет для обработки деревом

In [279]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [280]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

Возьмем небольшой срез данных для отладки, понимания и восстановления функции

In [281]:
debug_x = x[10:30]
debug_y = y[10:30]


In [282]:
f_x = debug_x[:,0]
logger.setLevel(logging.DEBUG)
my_clf.debug_threshold(f_x, debug_y)
logger.setLevel(logging.INFO)

DEBUG:decision tree:['sorted_x = ', array([ 0.01035186,  0.01965658,  0.02565568,  0.03442147,  0.04656027,
        0.05243609,  0.07542658,  0.16628408,  0.18686856,  0.20092338,
        0.22181277,  0.39224848,  0.39299459,  0.45251583,  0.54845806,
        0.60279441,  0.70407398,  0.96467256,  0.9999999 ,  0.9999999 ])]
DEBUG:decision tree:['sorted_y = ', array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int64)]
DEBUG:decision tree:['class number ', 2]
DEBUG:decision tree:['splitted_sorted_y = ', array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int64)]
DEBUG:decision tree:['r_border_ids = ', array([ 2,  3, 11, 12, 17, 18], dtype=int64)]
DEBUG:decision tree:['eq_el_count=', array([0, 1, 8, 1, 5, 1], dtype=int64)]
DEBUG:decision tree:['one_hot_code', array([[ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.]])]
DEBUG:decision tree:['class_increments', array([[ 2.,  0.],
       [ 0.,

In [283]:
f_x.argsort()

array([ 0,  2,  8, 14, 11, 13, 10,  4, 19,  7,  5, 12, 16, 15,  3,  6, 17,
        1, 18,  9], dtype=int64)

## Проверка скорости работы

In [284]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
logger.info(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
logger.info(t2 - t1)

INFO:decision tree:2.607527256011963
INFO:decision tree:1.5042858123779297


## Проверка качества работы

In [285]:
gkf = KFold(n_splits=5, shuffle=True)

In [286]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    logger.info(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

INFO:decision tree:0.999958426873
INFO:decision tree:1.0
INFO:decision tree:1.0
INFO:decision tree:0.999958426873
INFO:decision tree:1.0


In [287]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    logger.info(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

INFO:decision tree:0.891826723206
INFO:decision tree:0.891660430698
INFO:decision tree:0.893739087054
INFO:decision tree:0.893240209529
INFO:decision tree:0.891448052218


# Применить для задачи Titanic 

In [288]:
fn = 'train.csv'
df = pd.read_csv(fn)
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, MinMaxScaler, Imputer, LabelBinarizer, OneHotEncoder,StandardScaler
from sklearn.feature_extraction import DictVectorizer

def get_sex_col(df):
    lb= LabelBinarizer()
    return lb.fit_transform(df[['Sex']])

def get_num_cols(df):
    bins = [0, 10, 15, 25, 40, 55, 100]
    labels = [10, 15, 25, 40, 55, 100 ]
    mn = df['Age'].mean()
    df['AgeGrp'] = df['Age'].fillna(mn)
    df['AgeGrp'] = pd.cut(df['AgeGrp'], bins, labels=labels)

    bins = [0, 10, 30, 100, 600]
    labels = [10, 30, 100, 600 ]
    mn = df['Fare'].mean()
    df['FareGrp'] = df['Fare'].fillna(mn)
    df['FareGrp'] = pd.cut(df['FareGrp'], bins, labels=labels)    
    return df[['AgeGrp', 'FareGrp']]

def get_pclass_col(df):
    return df[['Pclass']]

def get_port_col(df):
    le = LabelEncoder()
    return le.fit_transform(df['Embarked'].fillna('S').T).reshape(-1, 1).astype('int')


def get_cabin_col(df):
    le = LabelEncoder()
    return le.fit_transform(df['Cabin'].fillna('NaN').T).reshape(-1, 1).astype('float')

# наличие родственников можно объединить
def get_rel_col(df):
    return np.sum(df[['SibSp','Parch']] , axis=1).values.reshape(-1, 1).astype('float')  

pipeline = make_union(*[
    make_pipeline(FunctionTransformer(get_num_cols, validate=False), Imputer(strategy='mean'), MinMaxScaler()),
    make_pipeline(FunctionTransformer(get_pclass_col, validate=False), OneHotEncoder(sparse=False)),
    make_pipeline(FunctionTransformer(get_sex_col, validate=False)),
    make_pipeline(FunctionTransformer(get_port_col, validate=False), OneHotEncoder(sparse=False)),
    make_pipeline(FunctionTransformer(get_cabin_col, validate=False), MinMaxScaler()),
    make_pipeline(FunctionTransformer(get_rel_col, validate=False), StandardScaler())
])


In [289]:
df_train = df.copy()
y_train = df_train['Survived']
print(y_train.shape)
x_train = pipeline.fit_transform(df_train)
print(x_train.shape)

(891,)
(891, 11)


In [290]:
# сделаем предсказание
def predict_and_save(model, df_test):
    x_test = pipeline.fit_transform(df_test) 
    y_test =  model.predict(x_test)
    df_predicted = pd.DataFrame({'PassengerId': df_test['PassengerId'], 'Survived': y_test})
    df_predicted.to_csv('sample_submission.csv', sep=',', index=False)

In [291]:
df_test = pd.read_csv('test.csv')


In [292]:
model = MyDecisionTreeClassifier(min_samples_split=2)
model.fit(x_train, y_train.as_matrix())
predict_and_save(model, df_test)

результат с каггла -  0.62679 судя по всему дерево нуждается в оптимизации

In [293]:
accuracy_score(y_train, model.predict(x_train))

0.61616161616161613